<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card%20predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Bidirectional

# GPU configuration (already provided)
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Enable memory growth for all GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [2]:
# CardEncodingLookup layer (already provided)
class CardEncodingLookup(tf.keras.layers.Layer):
    def __init__(self, encoding_path, **kwargs):
        super(CardEncodingLookup, self).__init__(**kwargs)
        self.encoding_path = encoding_path
        # Load encodings during initialization
        with open(encoding_path, "rb") as f:
            self.encodings = pickle.load(f)
        # Get list of card names
        self.card_names = list(self.encodings.keys())

        # Create a lookup table for card indices
        # CHANGE: Use 0 as default_value instead of -1
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(
                tf.constant(self.card_names, dtype=tf.string),
                tf.range(1, len(self.card_names) + 1, dtype=tf.int64),  # Start from 1
            ),
            default_value=0,  # Use 0 for unknown cards
        )

        # Add a special "unknown" card embedding at index 0
        unknown_card = {
            "colorid": np.zeros_like(self.encodings[self.card_names[0]]["colorid"]),
            "text": np.zeros_like(self.encodings[self.card_names[0]]["text"]),
            "types": np.zeros_like(self.encodings[self.card_names[0]]["types"]),
            "mana": np.zeros_like(self.encodings[self.card_names[0]]["mana"]),
            "stats": np.zeros_like(self.encodings[self.card_names[0]]["stats"]),
        }

        # Convert encodings to tensors and store them with the unknown card at index 0
        self.colorid_tensors = tf.constant(
            [unknown_card["colorid"]]
            + [self.encodings[name]["colorid"] for name in self.card_names]
        )
        self.text_tensors = tf.constant(
            [unknown_card["text"]]
            + [self.encodings[name]["text"] for name in self.card_names]
        )
        self.types_tensors = tf.constant(
            [unknown_card["types"]]
            + [self.encodings[name]["types"] for name in self.card_names]
        )
        self.mana_tensors = tf.constant(
            [unknown_card["mana"]]
            + [self.encodings[name]["mana"] for name in self.card_names]
        )
        self.stats_tensors = tf.constant(
            [unknown_card["stats"]]
            + [self.encodings[name]["stats"] for name in self.card_names]
        )

    def call(self, inputs):
        # Look up the indices for the card names
        indices = self.table.lookup(inputs)

        # Gather the encodings for the indices
        colorid = tf.gather(self.colorid_tensors, indices)
        text = tf.gather(self.text_tensors, indices)
        types = tf.gather(self.types_tensors, indices)
        mana = tf.gather(self.mana_tensors, indices)
        stats = tf.gather(self.stats_tensors, indices)

        return {
            "colorid": colorid,
            "text": text,
            "types": types,
            "mana": mana,
            "stats": stats,
        }

# Create the lookup layer (already defined in the provided code)
card_lookup = CardEncodingLookup(encoding_path="data/card_encodings.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'data/card_encodings.pkl'

In [ ]:
# 1. Create a model for synergy prediction
class SynergyModel(tf.keras.Model):
    def __init__(self, card_lookup, **kwargs):
        super(SynergyModel, self).__init__(**kwargs)
        self.card_lookup = card_lookup

        # Feature processing layers
        self.colorid_dense = tf.keras.layers.Dense(32, activation="leaky_relu")
        self.text_lstm = Bidirectional(
            tf.keras.layers.LSTM(
                64,
                return_sequences=True,
            )
        )
        self.text_lstm2 = Bidirectional(tf.keras.layers.LSTM(64))
        self.types_dense = tf.keras.layers.Dense(32, activation="leaky_relu")
        self.mana_dense = tf.keras.layers.Dense(32, activation="leaky_relu")
        self.stats_dense = tf.keras.layers.Dense(32, activation="leaky_relu")

        # Combination layers
        self.combine_dense1 = tf.keras.layers.Dense(256, activation="leaky_relu")
        self.combine_dense2 = tf.keras.layers.Dense(32, activation="leaky_relu")
        self.output_layer = tf.keras.layers.Dense(1)  # Single output for synergy score

    def process_card(self, card_encodings):
        # Process each feature type
        colorid_features = self.colorid_dense(card_encodings["colorid"])

        # Extract the text tensor for LSTM - this is the key fix
        text_tensor = card_encodings["text"]
        # Ensure it has the right shape for LSTM (batch_size, timesteps, features)
        text_features = self.text_lstm(text_tensor)
        text_features = self.text_lstm2(text_features)

        types_features = self.types_dense(card_encodings["types"])
        mana_features = self.mana_dense(card_encodings["mana"])
        stats_features = self.stats_dense(card_encodings["stats"])

        # Concatenate all features
        return tf.concat(
            [
                colorid_features,
                text_features,
                types_features,
                mana_features,
                stats_features,
            ],
            axis=1,
        )

    def call(self, inputs):
        card1_name, card2_name = inputs["card_1"], inputs["card_2"]

        # Get encodings for both cards
        card1_encodings = self.card_lookup(card1_name)
        card2_encodings = self.card_lookup(card2_name)

        # Process each card
        card1_features = self.process_card(card1_encodings)
        card2_features = self.process_card(card2_encodings)

        # Combine features from both cards
        combined_features = tf.concat([card1_features, card2_features], axis=1)

        # Final prediction layers
        x = self.combine_dense1(combined_features)
        x = self.combine_dense2(x)
        synergy = self.output_layer(x)

        return synergy

In [ ]:
# 2. Function to load and preprocess the CSV data
def load_synergy_data(csv_path, test_size=0.05, random_state=42):
    # Load the CSV file
    df = pd.read_csv(csv_path)

    # Get indices for train/test split
    indices = np.arange(len(df))
    indices_train, indices_test = train_test_split(
        indices, test_size=test_size, random_state=random_state
    )

    # Split the dataframe
    df_train = df.iloc[indices_train]
    df_test = df.iloc[indices_test]

    # Create input dictionaries and output arrays
    X_train = {
        "card_1": df_train["main_card"].values,
        "card_2": df_train["combo_card"].values,
    }
    X_test = {
        "card_1": df_test["main_card"].values,
        "card_2": df_test["combo_card"].values,
    }

    df_train["synergies_score"] = df_train["synergies_score"] * 0.05
    df_test["synergies_score"] = df_test["synergies_score"] * 0.05

    y_train = df_train["synergies_score"].values
    y_test = df_test["synergies_score"].values

    return X_train, X_test, y_train, y_test

# Load and preprocess data
# Assuming you have a CSV file with columns: card_1, card_2, synergy
X_train, X_test, y_train, y_test = load_synergy_data("/content/drive/MyDrive/synergy_dataset.csv")

In [ ]:
# 3. Function to train the model
def train_synergy_model(
    model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32
):
    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="mse",
        metrics=["mae"],
    )

    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(
        batch_size
    )
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)

    # Train the model
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=val_dataset,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=5, restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor="val_loss", factor=0.5, patience=3
            ),
        ],
    )

    return history


# 4. Function to evaluate and visualize results
def evaluate_model(model, X_test, y_test, history=None):
    # Create test dataset
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

    # Evaluate the model
    results = model.evaluate(test_dataset)
    print(f"Test Loss: {results[0]:.4f}")
    print(f"Test MAE: {results[1]:.4f}")

    # Make predictions
    y_pred = model.predict(X_test)

    # Plot predictions vs actual
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "r--")
    plt.xlabel("Actual Synergy")
    plt.ylabel("Predicted Synergy")
    plt.title("Synergy Prediction: Actual vs Predicted")
    plt.show()

    # Plot training history if available
    if history:
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title("Model Loss")
        plt.ylabel("Loss")
        plt.xlabel("Epoch")
        plt.legend(["Train", "Validation"], loc="upper right")

        plt.subplot(1, 2, 2)
        plt.plot(history.history["mae"])
        plt.plot(history.history["val_mae"])
        plt.title("Model MAE")
        plt.ylabel("MAE")
        plt.xlabel("Epoch")
        plt.legend(["Train", "Validation"], loc="upper right")
        plt.show()


# 5. Function to predict synergy for new card pairs
def predict_synergy(model, card1_name, card2_name):
    # Create input dictionary
    input_data = {"card_1": np.array([card1_name]), "card_2": np.array([card2_name])}

    # Make prediction
    synergy = model.predict(input_data)[0][0]

    return synergy

In [ ]:
# Create the synergy model
synergy_model = SynergyModel(card_lookup)

synergy_model.summary()



# Train the model
history = train_synergy_model(
    synergy_model,
    X_train,
    y_train,
    X_test,
    y_test,  # Using test set as validation for simplicity
    epochs=10,
    batch_size=1024,
)

synergy_model.save_weights("synergy_model.weights.h5")